In [12]:
import requests
import time

In [13]:
t0 = time.time()
# Example for posting a file to the application
url = "http://127.0.0.1:5000/upload_def_file"
file_name = "Dockerfile"
file_path = "Dockerfile"
my_files = {"file": (file_name, open(file_path,"rb"))}
response = requests.post(url, files=my_files)
print(response.text)
print(time.time() - t0)

3c65f418-b596-4115-ad8a-3f6c6b0acd7e
0.7232089042663574


In [14]:
t0 = time.time()
# Example for building a container
url = "http://127.0.0.1:5000/build"
my_data = {"container_id": response.text,
          "to_format": "docker", "container_name": "my_docker_test"}

response = requests.post(url, json=my_data)
print(response.text)
print(time.time() - t0)

3fc86336-b34b-4037-8a7b-3e54c13b1040
213.62782287597656


In [15]:
t0 = time.time()
import os
# Example of pulling down a container
url = "http://127.0.0.1:5000/pull"
my_data = {"build_id": response.text}
response = requests.get(url, json=my_data)
with open("my_container.tar", 'wb') as f:
    f.write(response.content)

print(os.path.getsize("my_container.tar"))
print(time.time() - t0)

965066752
28.214515924453735
